In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from time import sleep
from login import EMAIL
from login import PASSWORD

In [11]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)
driver.maximize_window()

driver.get("https://pro.urbanize.city/users/sign_in")

wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))


iframes = driver.find_elements(By.TAG_NAME, "iframe")
if iframes:
    driver.switch_to.frame(iframes[0])

email_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='email'], input[type='email']")))
email_input.send_keys(EMAIL)

password_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='password'], input[type='password']")))
password_input.send_keys(PASSWORD)

login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.prime[actionlogin]")))
login_button.click()


wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))



# ... rest of your code ...

# Don't forget to switch back to default content after interacting with the iframe
driver.switch_to.default_content()
wait.until(EC.visibility_of_element_located((By.XPATH, "//a[contains(@class, 'border-cyan-500') and contains(text(), 'Dashboard')]")))

driver.get("https://pro.urbanize.city/los_angeles/projects")
# wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "searchicon")))
print(driver.page_source)

<html><head><style type="text/css" data-tag-name="trix-editor">trix-editor {
  display: block;
}

trix-editor:empty:not(:focus)::before {
  content: attr(placeholder);
  color: graytext;
  cursor: text;
  pointer-events: none;
}

trix-editor a[contenteditable=false] {
  cursor: text;
}

trix-editor img {
  max-width: 100%;
  height: auto;
}

trix-editor [data-trix-attachment] figcaption textarea {
  resize: none;
}

trix-editor [data-trix-attachment] figcaption textarea.trix-autoresize-clone {
  position: absolute;
  left: -9999px;
  max-height: 0px;
}

trix-editor [data-trix-attachment] figcaption[data-trix-placeholder]:empty::before {
  content: attr(data-trix-placeholder);
  color: graytext;
}

trix-editor [data-trix-cursor-target] {
  display: inline-block !important;
  width: 1px !important;
  padding: 0 !important;
  margin: 0 !important;
  border: none !important;
}

trix-editor [data-trix-cursor-target=left] {
  vertical-align: top !important;
  margin-left: -1px !important;
}
